### Restart and Run All Cells

In [1]:
import calendar
import pandas as pd
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 3, 6)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

In [3]:
format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [4]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate


In [5]:
names = orders['name']
name = names.to_string(index=False)

In [6]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual


In [7]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'Series([], )'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market


### Create orders from stocks after adjust stocks in port_lite from consensus

In [8]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

0

In [9]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy

,trade,name,qty,price,active,reason,market
0,O,AMATA,0,0.00,0,,SET100
1,O,ASIAN,9000,15.20,0,,SET
2,O,ASK,3000,42.00,0,1L,SET
3,O,ASP,30000,3.70,0,6L,SET
4,O,BAM,10000,20.50,0,,SET100
5,O,BBL,2400,130.00,0,1L,SET50
6,O,BDMS,12000,23.20,0,,SET50
7,O,CHG,60000,3.36,0,,SET100
8,O,DCC,0,0.00,0,,SET
9,O,GLOBAL,9000,19.40,0,,SET50


In [10]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
df_sell = pd.read_sql(sql, conlite)
df_sell

,trade,name,qty,price,active,reason,market
0,I,JASIF,10000,11.20,0,DOS,SET
1,I,HREIT,40100,9.00,0,DOS,SET
2,I,RATCH,3000,48.00,0,5pct,SET50
3,I,BCH,5000,21.40,0,CP1S,SET100
4,I,DOHOME,4800,21.30,0,5pct,SET100
5,I,PTT,3000,41.25,0,5pct,SET50
6,I,SINGER,1500,48.50,0,15pct,SET100
7,I,IP,2500,21.20,0,5pct,mai


In [11]:
df_order = df_buy.append(df_sell, ignore_index=True)
df_order

,trade,name,qty,price,active,reason,market
0,O,AMATA,0,0.00,0,,SET100
1,O,ASIAN,9000,15.20,0,,SET
2,O,ASK,3000,42.00,0,1L,SET
3,O,ASP,30000,3.70,0,6L,SET
4,O,BAM,10000,20.50,0,,SET100
5,O,BBL,2400,130.00,0,1L,SET50
6,O,BDMS,12000,23.20,0,,SET50
7,O,CHG,60000,3.36,0,,SET100
8,O,DCC,0,0.00,0,,SET
9,O,GLOBAL,9000,19.40,0,,SET50


In [12]:
df_order.loc[df_order.trade == 'O','trade'] = 'B'
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,32,32,32,32,32,32
I,8,8,8,8,8,8


In [13]:
df_order.loc[df_order.trade == 'I','trade'] = 'S'
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,32,32,32,32,32,32
S,8,8,8,8,8,8


In [14]:
sql = '''
SELECT name, xdate
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend.shape

(78, 2)

In [15]:
df_merge = pd.merge(df_order, dividend, on='name', how='outer', indicator=True)
df_merge.shape

(78, 9)

### If there is no dividend record, must create dividend record before proceed

In [16]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge


In [17]:
mask = df_merge['_merge'] == 'both'
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,32,32,32,32,32,32,32
S,8,8,8,8,8,8,8


In [18]:
fm_date = pd.to_datetime('2022-02-07')
to_date = pd.to_datetime('2022-02-21')
df_orders.dtypes

trade      object
name       object
qty         int64
price     float64
active      int64
reason     object
market     object
xdate      object
dtype: object

In [19]:
fltr_buy = (df_orders.trade == 'B') & (df_orders.xdate >= '2022-03-08') & (df_orders.xdate <= '2022-03-21') 
df_buy = df_orders[fltr_buy]
df_buy.shape

(9, 8)

In [20]:
fltr_sell = (df_orders.trade == 'S')
df_sell = df_orders[fltr_sell]
df_sell.shape

(8, 8)

In [21]:
df_orders = df_buy.append(df_sell, ignore_index=True)
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders

,trade,name,qty,price,active,reason,market,xdate
0,B,ASP,30000,3.70,0,6L,SET,2022-03-10
1,B,BDMS,12000,23.20,0,,SET50,2022-03-09
2,B,IMH,9000,20.00,0,1L,mai,2022-03-18
3,B,MEGA,1500,40.50,0,10pct,SET100,2022-03-08
4,B,SYNEX,5000,22.50,0,CP1S,SET100,2022-03-10
5,B,TPIPL,60000,1.70,0,,SET,2022-03-09
6,B,TQM,4500,44.00,0,6L,SET100,2022-03-10
7,B,TU,9000,19.30,0,3L,SET50,2022-03-09
8,B,WHART,10000,12.10,0,DOS,SET,2022-03-09
9,S,JASIF,10000,11.20,0,DOS,SET,2022-03-03


In [22]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

In [23]:
rcds = df_orders.values.tolist()
len(rcds)

17

In [24]:
for rcd in rcds:
    print(rcd)

['B', 'ASP', 30000, 3.7, 0, '6L', 'SET', '2022-03-10']
['B', 'BDMS', 12000, 23.2, 0, '', 'SET50', '2022-03-09']
['B', 'IMH', 9000, 20.0, 0, '1L', 'mai', '2022-03-18']
['B', 'MEGA', 1500, 40.5, 0, '10pct', 'SET100', '2022-03-08']
['B', 'SYNEX', 5000, 22.5, 0, 'CP1S', 'SET100', '2022-03-10']
['B', 'TPIPL', 60000, 1.7, 0, '', 'SET', '2022-03-09']
['B', 'TQM', 4500, 44.0, 0, '6L', 'SET100', '2022-03-10']
['B', 'TU', 9000, 19.3, 0, '3L', 'SET50', '2022-03-09']
['B', 'WHART', 10000, 12.1, 0, 'DOS', 'SET', '2022-03-09']
['S', 'JASIF', 10000, 11.2, 0, 'DOS', 'SET', '2022-03-03']
['S', 'HREIT', 40100, 9.0, 0, 'DOS', 'SET', '2022-03-04']
['S', 'RATCH', 3000, 48.0, 0, '5pct', 'SET50', '2022-03-02']
['S', 'BCH', 5000, 21.4, 0, 'CP1S', 'SET100', '2022-05-10']
['S', 'DOHOME', 4800, 21.3, 0, '5pct', 'SET100', '2022-02-02']
['S', 'PTT', 3000, 41.25, 0, '5pct', 'SET50', '2022-03-03']
['S', 'SINGER', 1500, 48.5, 0, '15pct', 'SET100', '2022-04-29']
['S', 'IP', 2500, 21.2, 0, '5pct', 'mai', '2022-02-02']


In [25]:
sql = """
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [26]:
for rcd in rcds:
    conlite.execute(sql, rcd)

### After call Ord-log.bat

In [27]:
def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [28]:
df = pd.read_csv('../data/orders-log.csv')
#df.rename(columns = {'col1': 'name', 'col2': 'spd','col3': 'reason', 'col4': 'market',
#                     'col5': 'qty','col6': 'target','col7': 'current', 'col8': 'chg',
#                     'col9': 'percent', 'col10': 'active'}, inplace = True)
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

df_tab = pn.widgets.Tabulator(df, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [29]:
df[colt]

,trans,name,spd,reason,qty,target,current,percent,active,xdate
0,Buy,ASP,-6,6L,30000,3.70,3.82,-2.55%,1,NaN
1,Buy,BDMS,-12,NaN,12000,23.20,24.40,-0.81%,0,NaN
2,Buy,IMH,-7,1L,9000,20.00,20.70,-0.96%,1,NaN
3,Buy,MEGA,-13,10pct,1500,40.50,43.75,-1.69%,0,NaN
4,Buy,SYNEX,-28,CP1S,5000,22.50,25.75,-2.83%,0,NaN
5,Buy,TPIPL,-3,3L,60000,1.70,1.73,-0.57%,2,NaN
6,Buy,TQM,-4,6L,4500,44.00,45.00,+1.69%,1,NaN
7,Buy,TU,-4,3L,9000,19.30,19.70,-1.01%,1,NaN
8,Buy,WHART,-3,DOS,10000,12.10,12.40,+2.48%,1,NaN
9,Sell,BCH,16,CP1S,5000,21.40,19.80,-1.00%,0,NaN
